# Aligning a model on Arcee Cloud

In this notebook, you will learn how to align a model with Supervised Fine Tuning on Arcee Cloud.

In order to run this demo, you need a Starter account on Arcee Cloud. Please see our [pricing](https://www.arcee.ai/pricing) page for details.

The Arcee documentation is available at [docs.arcee.ai](https://docs.arcee.ai/deployment/start-deployment).

## Prerequisites

Please [sign up](https://app.arcee.ai/account/signup) to Arcee Cloud and create an [API key](https://docs.arcee.ai/getting-arcee-api-key/getting-arcee-api-key).

Then, please update the cell below with your API key. Remember to keep this key safe, and **DON'T COMMIT IT to one of your repositories**.

In [ ]:
%env ARCEE_API_KEY=YOUR_API_KEY

Create a new Python environment (optional but recommended) and install [arcee-python](https://github.com/arcee-ai/arcee-python).

In [ ]:
# Uncomment the next three lines to create a virtual environment
#!pip install -q virtualenv
#!virtualenv -q arcee-cloud
#!source arcee-cloud/bin/activate

%pip install -qU arcee-py

We will also need the Hugging Face [`datasets`](https://huggingface.co/datasets) library to download an alignment model from the Hugging Face hub.

In [ ]:
%pip install -q datasets

In [ ]:
import arcee
import pprint
import pandas as pd
from datasets import load_dataset

# Preparing our dataset

Model alignment requires a dataset with questions and answers. At the moment, Arcee Cloud requires this dataset to be in the CSV format, with one column for questions and one for answers.

Let's download the [`reasoning-sharegpt`](https://huggingface.co/datasets/arcee-ai/reasoning-sharegpt) dataset from the Hugging Face hub. 

This dataset contains close to 30,000 single-turn question-answers pairs focused on step-by-step reasoning.

In [ ]:
dataset_name = "arcee-ai/reasoning-sharegpt"

In [ ]:
dataset = load_dataset(dataset_name)
print(dataset)
pprint.pprint(dataset['train'][0])

Now, let's format it in two columns with just the questions and the answers, then save it to the CSV format.

In [ ]:
data = [{
    "questions": item['conversations'][0]['value'],
    "answers": item['conversations'][1]['value']
} for item in dataset['train']]

df = pd.DataFrame(data)
df.head()

The last preparation step is to save the dataset in CSV format.

In [ ]:
csv_name = "reasoning-sharegpt.csv"

df.to_csv(csv_name, index=False)

## Uploading our dataset

Now that we've prepared the dataset, we can upload it to Arcee Cloud with the `upload_qa_pairs_from_csv()` API.

In [ ]:
help(arcee.upload_qa_pairs_from_csv)

In [ ]:
qa_set="reasoning-share-gpt"

In [ ]:
arcee.upload_qa_pairs_from_csv(
  qa_set=qa_set,
  csv_path=f"./{csv_name}",
  question_column="questions",
  answer_column="answers"
)

# Aligning the model

We can now use the dataset for model alignment. 

Here, we will align the [Llama-3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B) model. We could pick any model available on the Hugging Face hub, or a model we've already worked with on Arcee Cloud.

Let's launch the alignment job with the `start_alignment()` API.

In [ ]:
help(arcee.start_alignment)

In [ ]:
alignment_name = "llama-3-8B-reasoning-share-gpt"

In [ ]:
response=arcee.start_alignment(alignment_name=alignment_name,
                      qa_set=qa_set,
                      hf_model="meta-llama/Meta-Llama-3-8B"
)
print(response)

In [ ]:
from time import sleep

while True:
    response = arcee.alignment_status(alignment_name)
    if response["processing_state"] == "processing":
        print("Alignment is in progress. Waiting 5 minutes before checking again.")
        sleep(300)
    else:
        print(response)
        break

## Deploying our aligned model

Once alignment is complete, we can deploy and test the aligned model. As part of the Arcee Cloud free tier, this is free of charge and the endpoint will be automatically shut down after 2 hours.

Deployment should take 5-7 minutes. Please see the model deployment sample notebook for details.

In [ ]:
deployment_name = "llama-3-8B-reasoning-share-gpt"

In [ ]:
response = arcee.start_deployment(deployment_name=deployment_name, alignment=alignment_name)

while True:
    response = arcee.deployment_status(deployment_name)
    if response["deployment_processing_state"] == "pending":
        print("Deployment is in progress. Waiting 30 seconds before checking again.")
        sleep(30)
    else:
        print(response)
        break

Once the model endpoint is up and running, we can prompt the model with a domain-specific question.

In [ ]:
query = "Give me instructions to change the oil on my Harley-Davidson."

response = arcee.generate(deployment_name=deployment_name, query=query)
print(response["text"])

## Stopping our deployment

When we're done working with our model, we should stop the deployment to save resources and avoid unwanted charges.

The `stop_deployment()` API only requires the deployment name.

In [ ]:
arcee.stop_deployment(deployment_name=deployment_name)
arcee.deployment_status(deployment_name)

This concludes the model alignment demonstration. Thank you for your time!

If you'd like to know more about using Arcee Cloud in your organization, please visit the [Arcee website](https://www.arcee.ai), or contact [sales@arcee.ai](mailto:sales@arcee.ai).
